# Import Lib

In [7]:
%matplotlib inline
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from random import randint
from scipy.io import mmread


# Import Graph

In [8]:

read_file = mmread('web-edu.mtx')
G = nx.Graph(read_file)

ValueError: not enough values to unpack (expected 5, got 3)

Try new visualization

In [ ]:
pos = nx.spring_layout(G, iterations=15, seed=1721)
fig, ax = plt.subplots(figsize=(15, 9))
ax.axis("off")
plot_options = {"node_size": 10, "with_labels": False, "width": 0.15}

nx.draw_networkx(G, pos=pos, ax=ax, **plot_options)

# Basic topological attributes

In [ ]:
def measure(G):
    print(f"Number of nodes: {G.number_of_nodes()}")
    print(f"Number of edges: {G.number_of_edges()}")
    print(f"Density:{nx.density(G)}")
    print(f"Transitivity: {nx.transitivity(G)}")
    print(f"Average Clustering: {nx.average_clustering(G)}")
    print(f"Max Degree: {max(dict(G.degree()).values())}")
    print(f"Min Degree: {min(dict(G.degree()).values())}")
    print(f"Average Degree: {np.mean(list(dict(G.degree()).values()))}")
    print(f"Number of components:{nx.number_connected_components(G)}")
    print(f"Assortativity: {nx.degree_assortativity_coefficient(G)}")

    if(nx.is_connected(G)):
        print("Graph is connected")
        print(f"Diameter:{nx.diameter(G)}")
    else:
        print("Graph is not connected")


In [ ]:
measure(G)

In [ ]:
betwenness_centrality = nx.betweenness_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
degree_centrality = nx.degree_centrality(G)

plt.figure(figsize=(15,5))
plt.subplot(131)
plt.title("Betweenness distribution")
plt.hist(betwenness_centrality.values(), bins=100)
plt.subplot(132)
plt.title("Closeness distribution")
plt.hist(closeness_centrality.values(), bins=100)
plt.subplot(133)
plt.title("Degree centrality distribution")
plt.hist(degree_centrality.values(), bins=100)
plt.show()

In [ ]:
shortest_path_lengths = dict(nx.all_pairs_shortest_path_length(G))

# Find the maximum index of all nodes in the graph
max_index = max(G.nodes())

# Create an array to store values from 0 up to (and including) the maximum shortest path length
path_lengths = np.zeros(max_index + 1, dtype=int)

# Extract the frequency of shortest path lengths between two nodes
for pls in shortest_path_lengths.values():
    pl, cnts = np.unique(list(pls.values()), return_counts=True)
    path_lengths[pl] += cnts

# Express frequency distribution as a percentage (ignoring path lengths of 0)
freq_percent = 100 * path_lengths[1:] / path_lengths[1:].sum()

# Plot the frequency distribution (ignoring path lengths of 0) as a percentage
fig, ax = plt.subplots(figsize=(4, 3))
ax.bar(np.arange(1, max_index + 1), height=freq_percent)
ax.set_title(
    "Distribution of shortest path length in G", loc="center"
)
ax.set_xlabel("Shortest Path Length")
ax.set_ylabel("Frequency (%)")

# Clustering Coeficient Histogram

In [ ]:
plt.figure(figsize=(4, 3))
plt.hist(nx.clustering(G).values(), bins=50)
plt.title("Clustering Coefficient Histogram ", loc="center")
plt.xlabel("Clustering Coefficient")
plt.ylabel("Counts")

In [ ]:
#number of unique triangle in the network
triangles_per_node = list(nx.triangles(G).values())
sum(
    triangles_per_node
) / 3  # divide by 3 because each triangle is counted once for each node

In [ ]:
#average number of triangles 
np.mean(triangles_per_node)


# Bridges

In [ ]:
nx.has_bridges(G)


In [ ]:
bridges = list(nx.bridges(G))
len(bridges)

In [ ]:
local_bridges = list(nx.local_bridges(G, with_span=False))
len(local_bridges)

In [ ]:
plt.figure(figsize=(15, 8))
nx.draw_networkx(G, pos=pos, node_size=10, with_labels=False, width=0.15)
nx.draw_networkx_edges(
    G, pos, edgelist=local_bridges, width=0.5, edge_color="lawngreen"
)  # green color for local bridges
nx.draw_networkx_edges(
    G, pos, edgelist=bridges, width=0.5, edge_color="r"
)  # red color for bridges
plt.axis("off")

# Communities

In [ ]:
colors = ["" for x in range(G.number_of_nodes())]  # initialize colors list
counter = 0
for com in nx.community.label_propagation_communities(G):
    color = "#%06X" % randint(0, 0xFFFFFF)  # creates random RGB color
    counter += 1
    for node in list(
        com
    ):  # fill colors list with the particular color for the community nodes
        colors[node] = color
print(f"Number of Communities:{counter}")

In [ ]:
plt.figure(figsize=(15, 9))
plt.axis("off")
nx.draw_networkx(
    G, pos=pos, node_size=10, with_labels=False, width=0.15, node_color=colors
)

In [ ]:
# only 10 communities
colors = ["" for x in range(G.number_of_nodes())]
for com in nx.community.asyn_fluidc(G, 10, seed=0):
    color = "#%06X" % randint(0, 0xFFFFFF)  # creates random RGB color
    for node in list(com):
        colors[node] = color
plt.figure(figsize=(15, 9))
plt.axis("off")
nx.draw_networkx(
    G, pos=pos, node_size=10, with_labels=False, width=0.15, node_color=colors
)